# Crafter VLM GEPA Demo

This demo runs GEPA prompt optimization for a Crafter vision-language agent that uses image-only observations.

**What this demo does:**
1. Creates a local task app for the Crafter VLM agent
2. Runs GEPA prompt optimization to find the best system prompt
3. Extracts the optimized prompt from results
4. Runs eval jobs comparing baseline vs optimized prompts
5. Displays comparison results

In [ ]:
#
 
P
a
r
a
m
e
t
e
r
s
 
(
c
a
n
 
b
e
 
o
v
e
r
r
i
d
d
e
n
 
b
y
 
p
a
p
e
r
m
i
l
l
)

B
A
C
K
E
N
D
_
U
R
L
 
=
 
"
h
t
t
p
s
:
/
/
a
p
i
.
u
s
e
s
y
n
t
h
.
a
i
"
 
 
#
 
D
e
f
a
u
l
t
 
t
o
 
p
r
o
d
u
c
t
i
o
n

A
P
I
_
K
E
Y
 
=
 
N
o
n
e
 
 
#
 
W
i
l
l
 
b
e
 
s
e
t
 
b
a
s
e
d
 
o
n
 
e
n
v
i
r
o
n
m
e
n
t

P
O
L
I
C
Y
_
M
O
D
E
L
 
=
 
"
g
p
t
-
4
.
1
-
n
a
n
o
"
 
 
#
 
V
L
M
 
m
o
d
e
l
 
f
o
r
 
t
h
e
 
a
g
e
n
t

V
E
R
I
F
I
E
R
_
M
O
D
E
L
 
=
 
"
g
p
t
-
5
-
n
a
n
o
"
 
 
#
 
M
o
d
e
l
 
f
o
r
 
v
e
r
i
f
i
c
a
t
i
o
n
 
(
m
u
s
t
 
b
e
 
i
n
 
a
l
l
o
w
e
d
 
l
i
s
t
)

R
O
L
L
O
U
T
_
B
U
D
G
E
T
 
=
 
3
0
 
 
#
 
T
o
t
a
l
 
r
o
l
l
o
u
t
 
b
u
d
g
e
t

N
U
M
_
G
E
N
E
R
A
T
I
O
N
S
 
=
 
2
 
 
#
 
N
u
m
b
e
r
 
o
f
 
G
E
P
A
 
g
e
n
e
r
a
t
i
o
n
s

U
S
E
_
T
U
N
N
E
L
 
=
 
T
r
u
e
 
 
#
 
W
h
e
t
h
e
r
 
t
o
 
u
s
e
 
c
l
o
u
d
f
l
a
r
e
d
 
t
u
n
n
e
l
s
 
(
r
e
q
u
i
r
e
d
 
f
o
r
 
p
r
o
d
)

In [ ]:
#
 
S
t
e
p
 
1
:
 
I
m
p
o
r
t
s
 
a
n
d
 
S
e
t
u
p

f
r
o
m
 
_
_
f
u
t
u
r
e
_
_
 
i
m
p
o
r
t
 
a
n
n
o
t
a
t
i
o
n
s


i
m
p
o
r
t
 
a
s
y
n
c
i
o

i
m
p
o
r
t
 
j
s
o
n

i
m
p
o
r
t
 
o
s

i
m
p
o
r
t
 
s
y
s

i
m
p
o
r
t
 
t
i
m
e

f
r
o
m
 
p
a
t
h
l
i
b
 
i
m
p
o
r
t
 
P
a
t
h

f
r
o
m
 
t
y
p
i
n
g
 
i
m
p
o
r
t
 
A
n
y
,
 
D
i
c
t
,
 
L
i
s
t
,
 
O
p
t
i
o
n
a
l


i
m
p
o
r
t
 
h
t
t
p
x

f
r
o
m
 
d
o
t
e
n
v
 
i
m
p
o
r
t
 
l
o
a
d
_
d
o
t
e
n
v

f
r
o
m
 
o
p
e
n
a
i
 
i
m
p
o
r
t
 
A
s
y
n
c
O
p
e
n
A
I


l
o
a
d
_
d
o
t
e
n
v
(
)


#
 
A
d
d
 
p
a
r
e
n
t
 
d
i
r
e
c
t
o
r
y
 
t
o
 
p
a
t
h
 
f
o
r
 
i
m
p
o
r
t
s

s
y
s
.
p
a
t
h
.
i
n
s
e
r
t
(
0
,
 
s
t
r
(
P
a
t
h
(
'
.
'
)
.
r
e
s
o
l
v
e
(
)
.
p
a
r
e
n
t
.
p
a
r
e
n
t
)
)


f
r
o
m
 
s
y
n
t
h
_
a
i
.
s
d
k
.
a
p
i
.
e
v
a
l
 
i
m
p
o
r
t
 
E
v
a
l
J
o
b
,
 
E
v
a
l
J
o
b
C
o
n
f
i
g

f
r
o
m
 
s
y
n
t
h
_
a
i
.
s
d
k
.
a
p
i
.
t
r
a
i
n
.
p
r
o
m
p
t
_
l
e
a
r
n
i
n
g
 
i
m
p
o
r
t
 
P
r
o
m
p
t
L
e
a
r
n
i
n
g
J
o
b

f
r
o
m
 
s
y
n
t
h
_
a
i
.
s
d
k
.
l
e
a
r
n
i
n
g
.
p
r
o
m
p
t
_
l
e
a
r
n
i
n
g
_
c
l
i
e
n
t
 
i
m
p
o
r
t
 
P
r
o
m
p
t
L
e
a
r
n
i
n
g
C
l
i
e
n
t

f
r
o
m
 
s
y
n
t
h
_
a
i
.
s
d
k
.
l
e
a
r
n
i
n
g
.
r
l
 
i
m
p
o
r
t
 
m
i
n
t
_
e
n
v
i
r
o
n
m
e
n
t
_
a
p
i
_
k
e
y
,
 
s
e
t
u
p
_
e
n
v
i
r
o
n
m
e
n
t
_
a
p
i
_
k
e
y

f
r
o
m
 
s
y
n
t
h
_
a
i
.
s
d
k
.
l
o
c
a
l
a
p
i
 
i
m
p
o
r
t
 
L
o
c
a
l
A
P
I
C
o
n
f
i
g
,
 
c
r
e
a
t
e
_
l
o
c
a
l
_
a
p
i

f
r
o
m
 
s
y
n
t
h
_
a
i
.
s
d
k
.
t
a
s
k
 
i
m
p
o
r
t
 
T
a
s
k
I
n
f
o
,
 
r
u
n
_
s
e
r
v
e
r
_
b
a
c
k
g
r
o
u
n
d

f
r
o
m
 
s
y
n
t
h
_
a
i
.
s
d
k
.
t
a
s
k
.
c
o
n
t
r
a
c
t
s
 
i
m
p
o
r
t
 
R
o
l
l
o
u
t
M
e
t
r
i
c
s
,
 
R
o
l
l
o
u
t
R
e
q
u
e
s
t
,
 
R
o
l
l
o
u
t
R
e
s
p
o
n
s
e

f
r
o
m
 
s
y
n
t
h
_
a
i
.
s
d
k
.
t
u
n
n
e
l
s
 
i
m
p
o
r
t
 
w
a
i
t
_
f
o
r
_
h
e
a
l
t
h
_
c
h
e
c
k

f
r
o
m
 
s
y
n
t
h
_
a
i
.
s
d
k
.
t
u
n
n
e
l
s
.
t
u
n
n
e
l
e
d
_
a
p
i
 
i
m
p
o
r
t
 
T
u
n
n
e
l
e
d
L
o
c
a
l
A
P
I
,
 
T
u
n
n
e
l
B
a
c
k
e
n
d


f
r
o
m
 
c
r
a
f
t
e
r
_
l
o
g
i
c
 
i
m
p
o
r
t
 
(

 
 
 
 
A
C
T
I
O
N
_
S
T
R
I
N
G
_
T
O
_
I
N
T
,

 
 
 
 
C
R
A
F
T
E
R
_
A
L
L
O
W
E
D
_
A
C
T
I
O
N
S
,

 
 
 
 
C
r
a
f
t
e
r
E
n
v
i
r
o
n
m
e
n
t
W
r
a
p
p
e
r
,

 
 
 
 
C
r
a
f
t
e
r
S
c
o
r
e
r
,

 
 
 
 
C
r
a
f
t
e
r
V
L
M
R
e
A
c
t
P
o
l
i
c
y
,

 
 
 
 
n
o
r
m
a
l
i
z
e
_
a
c
t
i
o
n
_
n
a
m
e
,

)


p
r
i
n
t
(
'
I
m
p
o
r
t
s
 
l
o
a
d
e
d
 
s
u
c
c
e
s
s
f
u
l
l
y
'
)

In [ ]:
# Step 2: Configuration

SYNTH_API_BASE = 'https://api.usesynth.ai'
LOCAL_API_PORT = 8001
OPTIMIZED_LOCAL_API_PORT = 8002

print(f'Backend: {SYNTH_API_BASE}')
print(f'Local API Ports: {LOCAL_API_PORT}, {OPTIMIZED_LOCAL_API_PORT}')

In [ ]:
# Step 3: Get API Key and Check Backend Health

if API_KEY:
    SYNTH_API_KEY = API_KEY
else:
    SYNTH_API_KEY = os.environ.get('SYNTH_API_KEY', '').strip()

if not SYNTH_API_KEY:
    raise RuntimeError('SYNTH_API_KEY not set. Please set it in environment or pass as parameter.')

print(f'Using API Key: {SYNTH_API_KEY[:20]}...')

# Check backend health
r = httpx.get(f'{SYNTH_API_BASE}/health', timeout=30)
if r.status_code == 200:
    print(f'Backend health: {r.json()}')
else:
    raise RuntimeError(f'Backend not healthy: status {r.status_code}')

In [ ]:
#
 
S
t
e
p
 
5
:
 
L
o
c
a
l
 
A
P
I
 
F
a
c
t
o
r
y


A
P
P
_
I
D
 
=
 
"
c
r
a
f
t
e
r
_
v
l
m
"

A
P
P
_
N
A
M
E
 
=
 
"
C
r
a
f
t
e
r
 
V
L
M
 
R
e
A
c
t
 
A
g
e
n
t
"

T
O
O
L
_
N
A
M
E
 
=
 
"
c
r
a
f
t
e
r
_
i
n
t
e
r
a
c
t
"


d
e
f
 
c
r
e
a
t
e
_
c
r
a
f
t
e
r
_
v
l
m
_
l
o
c
a
l
_
a
p
i
(
s
y
s
t
e
m
_
p
r
o
m
p
t
:
 
s
t
r
,
 
e
n
v
_
a
p
i
_
k
e
y
:
 
s
t
r
)
:

 
 
 
 
"
"
"
F
a
c
t
o
r
y
 
t
o
 
c
r
e
a
t
e
 
a
 
C
r
a
f
t
e
r
 
V
L
M
 
t
a
s
k
 
a
p
p
 
w
i
t
h
 
a
 
s
p
e
c
i
f
i
c
 
s
y
s
t
e
m
 
p
r
o
m
p
t
.
"
"
"

 
 
 
 
#
 
I
m
p
o
r
t
 
i
n
s
i
d
e
 
f
a
c
t
o
r
y
 
t
o
 
e
n
s
u
r
e
 
a
v
a
i
l
a
b
i
l
i
t
y
 
i
n
 
c
l
o
s
u
r
e

 
 
 
 
f
r
o
m
 
c
r
a
f
t
e
r
_
l
o
g
i
c
 
i
m
p
o
r
t
 
(

 
 
 
 
 
 
 
 
A
C
T
I
O
N
_
S
T
R
I
N
G
_
T
O
_
I
N
T
,

 
 
 
 
 
 
 
 
C
R
A
F
T
E
R
_
A
L
L
O
W
E
D
_
A
C
T
I
O
N
S
,

 
 
 
 
 
 
 
 
C
r
a
f
t
e
r
E
n
v
i
r
o
n
m
e
n
t
W
r
a
p
p
e
r
,

 
 
 
 
 
 
 
 
C
r
a
f
t
e
r
S
c
o
r
e
r
,

 
 
 
 
 
 
 
 
C
r
a
f
t
e
r
V
L
M
R
e
A
c
t
P
o
l
i
c
y
,

 
 
 
 
 
 
 
 
n
o
r
m
a
l
i
z
e
_
a
c
t
i
o
n
_
n
a
m
e
,

 
 
 
 
)

 
 
 
 

 
 
 
 
o
s
.
e
n
v
i
r
o
n
[
'
E
N
V
I
R
O
N
M
E
N
T
_
A
P
I
_
K
E
Y
'
]
 
=
 
e
n
v
_
a
p
i
_
k
e
y


 
 
 
 
a
s
y
n
c
 
d
e
f
 
r
u
n
_
r
o
l
l
o
u
t
(
r
e
q
u
e
s
t
:
 
R
o
l
l
o
u
t
R
e
q
u
e
s
t
,
 
f
a
s
t
a
p
i
_
r
e
q
u
e
s
t
)
 
-
>
 
R
o
l
l
o
u
t
R
e
s
p
o
n
s
e
:

 
 
 
 
 
 
 
 
p
o
l
i
c
y
_
c
o
n
f
i
g
 
=
 
r
e
q
u
e
s
t
.
p
o
l
i
c
y
.
c
o
n
f
i
g
 
o
r
 
{
}

 
 
 
 
 
 
 
 
s
e
e
d
 
=
 
r
e
q
u
e
s
t
.
e
n
v
.
s
e
e
d
 
o
r
 
0

 
 
 
 
 
 
 
 
e
n
v
_
c
o
n
f
i
g
 
=
 
r
e
q
u
e
s
t
.
e
n
v
.
c
o
n
f
i
g
 
o
r
 
{
}

 
 
 
 
 
 
 
 
m
a
x
_
s
t
e
p
s
 
=
 
i
n
t
(
e
n
v
_
c
o
n
f
i
g
.
g
e
t
(
'
m
a
x
_
s
t
e
p
s
_
p
e
r
_
e
p
i
s
o
d
e
'
,
 
2
0
0
)
)

 
 
 
 
 
 
 
 
m
a
x
_
t
u
r
n
s
 
=
 
i
n
t
(
e
n
v
_
c
o
n
f
i
g
.
g
e
t
(
'
m
a
x
_
t
u
r
n
s
'
,
 
5
0
)
)


 
 
 
 
 
 
 
 
e
n
v
 
=
 
C
r
a
f
t
e
r
E
n
v
i
r
o
n
m
e
n
t
W
r
a
p
p
e
r
(
s
e
e
d
=
s
e
e
d
,
 
m
a
x
_
s
t
e
p
s
=
m
a
x
_
s
t
e
p
s
)

 
 
 
 
 
 
 
 
o
b
s
e
r
v
a
t
i
o
n
 
=
 
a
w
a
i
t
 
e
n
v
.
r
e
s
e
t
(
)


 
 
 
 
 
 
 
 
p
o
l
i
c
y
 
=
 
C
r
a
f
t
e
r
V
L
M
R
e
A
c
t
P
o
l
i
c
y
(

 
 
 
 
 
 
 
 
 
 
 
 
s
y
s
t
e
m
_
p
r
o
m
p
t
=
s
y
s
t
e
m
_
p
r
o
m
p
t
,

 
 
 
 
 
 
 
 
 
 
 
 
u
s
e
_
v
i
s
i
o
n
=
T
r
u
e
,

 
 
 
 
 
 
 
 
 
 
 
 
i
m
a
g
e
_
o
n
l
y
_
m
o
d
e
=
T
r
u
e
,

 
 
 
 
 
 
 
 
)


 
 
 
 
 
 
 
 
#
 
R
o
u
t
e
 
O
p
e
n
A
I
 
c
a
l
l
s
 
t
h
r
o
u
g
h
 
S
y
n
t
h
'
s
 
i
n
f
e
r
e
n
c
e
 
p
r
o
x
y
 
f
o
r
 
t
r
a
c
e
 
r
e
c
o
n
s
t
r
u
c
t
i
o
n

 
 
 
 
 
 
 
 
i
n
f
e
r
e
n
c
e
_
u
r
l
 
=
 
p
o
l
i
c
y
_
c
o
n
f
i
g
.
g
e
t
(
'
i
n
f
e
r
e
n
c
e
_
u
r
l
'
,
 
'
'
)

 
 
 
 
 
 
 
 
i
f
 
i
n
f
e
r
e
n
c
e
_
u
r
l
:

 
 
 
 
 
 
 
 
 
 
 
 
o
s
.
e
n
v
i
r
o
n
[
'
O
P
E
N
A
I
_
B
A
S
E
_
U
R
L
'
]
 
=
 
i
n
f
e
r
e
n
c
e
_
u
r
l

 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
#
 
U
s
e
 
p
o
l
i
c
y
_
c
o
n
f
i
g
 
a
p
i
_
k
e
y
 
(
f
r
o
m
 
S
y
n
t
h
 
p
r
o
x
y
)
 
o
r
 
f
a
l
l
 
b
a
c
k
 
t
o
 
O
P
E
N
A
I
_
A
P
I
_
K
E
Y
 
e
n
v
 
v
a
r

 
 
 
 
 
 
 
 
a
p
i
_
k
e
y
 
=
 
p
o
l
i
c
y
_
c
o
n
f
i
g
.
g
e
t
(
'
a
p
i
_
k
e
y
'
)
 
o
r
 
o
s
.
e
n
v
i
r
o
n
.
g
e
t
(
'
O
P
E
N
A
I
_
A
P
I
_
K
E
Y
'
)

 
 
 
 
 
 
 
 
i
f
 
n
o
t
 
a
p
i
_
k
e
y
:

 
 
 
 
 
 
 
 
 
 
 
 
r
a
i
s
e
 
V
a
l
u
e
E
r
r
o
r
(
"
N
o
 
A
P
I
 
k
e
y
 
a
v
a
i
l
a
b
l
e
:
 
p
o
l
i
c
y
_
c
o
n
f
i
g
[
'
a
p
i
_
k
e
y
'
]
 
a
n
d
 
O
P
E
N
A
I
_
A
P
I
_
K
E
Y
 
e
n
v
 
v
a
r
 
a
r
e
 
b
o
t
h
 
e
m
p
t
y
"
)

 
 
 
 
 
 
 
 
c
l
i
e
n
t
 
=
 
A
s
y
n
c
O
p
e
n
A
I
(
a
p
i
_
k
e
y
=
a
p
i
_
k
e
y
)


 
 
 
 
 
 
 
 
h
i
s
t
o
r
y
:
 
L
i
s
t
[
D
i
c
t
[
s
t
r
,
 
A
n
y
]
]
 
=
 
[
]

 
 
 
 
 
 
 
 
e
p
i
s
o
d
e
_
r
e
w
a
r
d
s
:
 
L
i
s
t
[
f
l
o
a
t
]
 
=
 
[
]


 
 
 
 
 
 
 
 
f
o
r
 
t
u
r
n
 
i
n
 
r
a
n
g
e
(
m
a
x
_
t
u
r
n
s
)
:

 
 
 
 
 
 
 
 
 
 
 
 
m
e
s
s
a
g
e
s
 
=
 
p
o
l
i
c
y
.
b
u
i
l
d
_
m
e
s
s
a
g
e
s
(
o
b
s
e
r
v
a
t
i
o
n
,
 
h
i
s
t
o
r
y
)

 
 
 
 
 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
 
 
 
 
r
e
s
p
o
n
s
e
 
=
 
a
w
a
i
t
 
c
l
i
e
n
t
.
c
h
a
t
.
c
o
m
p
l
e
t
i
o
n
s
.
c
r
e
a
t
e
(

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
m
o
d
e
l
=
p
o
l
i
c
y
_
c
o
n
f
i
g
.
g
e
t
(
'
m
o
d
e
l
'
,
 
P
O
L
I
C
Y
_
M
O
D
E
L
)
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
m
e
s
s
a
g
e
s
=
m
e
s
s
a
g
e
s
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
t
o
o
l
s
=
p
o
l
i
c
y
.
t
o
o
l
s
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
t
o
o
l
_
c
h
o
i
c
e
=
'
r
e
q
u
i
r
e
d
'
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
m
a
x
_
c
o
m
p
l
e
t
i
o
n
_
t
o
k
e
n
s
=
p
o
l
i
c
y
_
c
o
n
f
i
g
.
g
e
t
(
'
m
a
x
_
c
o
m
p
l
e
t
i
o
n
_
t
o
k
e
n
s
'
,
 
5
1
2
)
,

 
 
 
 
 
 
 
 
 
 
 
 
)

 
 
 
 
 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
 
 
 
 
m
e
s
s
a
g
e
 
=
 
r
e
s
p
o
n
s
e
.
c
h
o
i
c
e
s
[
0
]
.
m
e
s
s
a
g
e

 
 
 
 
 
 
 
 
 
 
 
 
r
e
s
p
o
n
s
e
_
t
e
x
t
 
=
 
m
e
s
s
a
g
e
.
c
o
n
t
e
n
t
 
o
r
 
'
'

 
 
 
 
 
 
 
 
 
 
 
 
t
o
o
l
_
c
a
l
l
s
 
=
 
[

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
{
'
i
d
'
:
 
t
c
.
i
d
,
 
'
t
y
p
e
'
:
 
'
f
u
n
c
t
i
o
n
'
,
 
'
f
u
n
c
t
i
o
n
'
:
 
{
'
n
a
m
e
'
:
 
t
c
.
f
u
n
c
t
i
o
n
.
n
a
m
e
,
 
'
a
r
g
u
m
e
n
t
s
'
:
 
t
c
.
f
u
n
c
t
i
o
n
.
a
r
g
u
m
e
n
t
s
}
}

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
t
c
 
i
n
 
(
m
e
s
s
a
g
e
.
t
o
o
l
_
c
a
l
l
s
 
o
r
 
[
]
)

 
 
 
 
 
 
 
 
 
 
 
 
]


 
 
 
 
 
 
 
 
 
 
 
 
n
e
x
t
_
o
b
s
e
r
v
a
t
i
o
n
 
=
 
o
b
s
e
r
v
a
t
i
o
n

 
 
 
 
 
 
 
 
 
 
 
 
t
o
o
l
_
r
e
s
p
o
n
s
e
s
:
 
L
i
s
t
[
D
i
c
t
[
s
t
r
,
 
A
n
y
]
]
 
=
 
[
]

 
 
 
 
 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
t
o
o
l
_
c
a
l
l
s
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
t
c
 
i
n
 
t
o
o
l
_
c
a
l
l
s
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
t
o
o
l
_
c
a
l
l
_
i
d
 
=
 
t
c
[
'
i
d
'
]

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
t
o
o
l
_
n
a
m
e
 
=
 
t
c
[
'
f
u
n
c
t
i
o
n
'
]
[
'
n
a
m
e
'
]

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
a
c
t
i
o
n
s
_
l
i
s
t
:
 
L
i
s
t
[
s
t
r
]
 
=
 
[
]

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
t
o
o
l
_
n
a
m
e
 
=
=
 
T
O
O
L
_
N
A
M
E
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
t
r
y
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
a
r
g
s
 
=
 
j
s
o
n
.
l
o
a
d
s
(
t
c
[
'
f
u
n
c
t
i
o
n
'
]
[
'
a
r
g
u
m
e
n
t
s
'
]
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
r
a
w
_
a
c
t
i
o
n
s
 
=
 
a
r
g
s
.
g
e
t
(
'
a
c
t
i
o
n
s
_
l
i
s
t
'
,
 
[
]
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
a
c
t
i
o
n
s
_
l
i
s
t
 
=
 
[
s
t
r
(
a
)
 
f
o
r
 
a
 
i
n
 
r
a
w
_
a
c
t
i
o
n
s
 
i
f
 
s
t
r
(
a
)
.
s
t
r
i
p
(
)
]
[
:
5
]

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
e
x
c
e
p
t
 
E
x
c
e
p
t
i
o
n
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
a
s
s

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
n
o
t
 
a
c
t
i
o
n
s
_
l
i
s
t
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
a
c
t
i
o
n
s
_
l
i
s
t
 
=
 
[
'
n
o
o
p
'
]


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
n
o
r
m
a
l
i
z
e
d
_
a
c
t
i
o
n
s
 
=
 
[
]

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
a
c
t
i
o
n
_
r
e
s
u
l
t
s
 
=
 
[
]


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
a
c
t
i
o
n
_
s
t
r
 
i
n
 
a
c
t
i
o
n
s
_
l
i
s
t
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
n
o
r
m
a
l
i
z
e
d
 
=
 
n
o
r
m
a
l
i
z
e
_
a
c
t
i
o
n
_
n
a
m
e
(
a
c
t
i
o
n
_
s
t
r
)
 
o
r
 
'
n
o
o
p
'

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
n
o
r
m
a
l
i
z
e
d
_
a
c
t
i
o
n
s
.
a
p
p
e
n
d
(
n
o
r
m
a
l
i
z
e
d
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
a
c
t
i
o
n
 
=
 
A
C
T
I
O
N
_
S
T
R
I
N
G
_
T
O
_
I
N
T
.
g
e
t
(
n
o
r
m
a
l
i
z
e
d
,
 
0
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
n
e
x
t
_
o
b
s
e
r
v
a
t
i
o
n
 
=
 
a
w
a
i
t
 
e
n
v
.
s
t
e
p
(
a
c
t
i
o
n
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
r
e
w
a
r
d
 
=
 
n
e
x
t
_
o
b
s
e
r
v
a
t
i
o
n
.
g
e
t
(
'
r
e
w
a
r
d
'
,
 
0
.
0
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
e
p
i
s
o
d
e
_
r
e
w
a
r
d
s
.
a
p
p
e
n
d
(
f
l
o
a
t
(
r
e
w
a
r
d
)
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
a
c
t
i
o
n
_
r
e
s
u
l
t
s
.
a
p
p
e
n
d
(
{

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
a
c
t
i
o
n
'
:
 
n
o
r
m
a
l
i
z
e
d
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
r
e
w
a
r
d
'
:
 
r
e
w
a
r
d
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
t
e
r
m
i
n
a
t
e
d
'
:
 
n
e
x
t
_
o
b
s
e
r
v
a
t
i
o
n
.
g
e
t
(
'
t
e
r
m
i
n
a
t
e
d
'
)
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
t
r
u
n
c
a
t
e
d
'
:
 
n
e
x
t
_
o
b
s
e
r
v
a
t
i
o
n
.
g
e
t
(
'
t
r
u
n
c
a
t
e
d
'
)
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
}
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
n
e
x
t
_
o
b
s
e
r
v
a
t
i
o
n
.
g
e
t
(
'
t
e
r
m
i
n
a
t
e
d
'
)
 
o
r
 
n
e
x
t
_
o
b
s
e
r
v
a
t
i
o
n
.
g
e
t
(
'
t
r
u
n
c
a
t
e
d
'
)
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
b
r
e
a
k


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
t
o
o
l
_
r
e
s
p
o
n
s
e
s
.
a
p
p
e
n
d
(
{
'
t
o
o
l
_
c
a
l
l
_
i
d
'
:
 
t
o
o
l
_
c
a
l
l
_
i
d
,
 
'
a
c
t
i
o
n
s
'
:
 
n
o
r
m
a
l
i
z
e
d
_
a
c
t
i
o
n
s
,
 
'
r
e
s
u
l
t
s
'
:
 
a
c
t
i
o
n
_
r
e
s
u
l
t
s
}
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
n
e
x
t
_
o
b
s
e
r
v
a
t
i
o
n
.
g
e
t
(
'
t
e
r
m
i
n
a
t
e
d
'
)
 
o
r
 
n
e
x
t
_
o
b
s
e
r
v
a
t
i
o
n
.
g
e
t
(
'
t
r
u
n
c
a
t
e
d
'
)
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
b
r
e
a
k

 
 
 
 
 
 
 
 
 
 
 
 
e
l
s
e
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
n
e
x
t
_
o
b
s
e
r
v
a
t
i
o
n
 
=
 
a
w
a
i
t
 
e
n
v
.
s
t
e
p
(
0
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
e
p
i
s
o
d
e
_
r
e
w
a
r
d
s
.
a
p
p
e
n
d
(
f
l
o
a
t
(
n
e
x
t
_
o
b
s
e
r
v
a
t
i
o
n
.
g
e
t
(
'
r
e
w
a
r
d
'
,
 
0
.
0
)
)
)


 
 
 
 
 
 
 
 
 
 
 
 
h
i
s
t
o
r
y
.
a
p
p
e
n
d
(
{
'
r
o
l
e
'
:
 
'
a
s
s
i
s
t
a
n
t
'
,
 
'
c
o
n
t
e
n
t
'
:
 
r
e
s
p
o
n
s
e
_
t
e
x
t
,
 
'
t
o
o
l
_
c
a
l
l
s
'
:
 
t
o
o
l
_
c
a
l
l
s
}
)

 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
r
e
s
p
 
i
n
 
t
o
o
l
_
r
e
s
p
o
n
s
e
s
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
h
i
s
t
o
r
y
.
a
p
p
e
n
d
(
{

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
r
o
l
e
'
:
 
'
t
o
o
l
'
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
t
o
o
l
_
c
a
l
l
_
i
d
'
:
 
r
e
s
p
[
'
t
o
o
l
_
c
a
l
l
_
i
d
'
]
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
c
o
n
t
e
n
t
'
:
 
j
s
o
n
.
d
u
m
p
s
(
{
'
a
c
t
i
o
n
s
'
:
 
r
e
s
p
[
'
a
c
t
i
o
n
s
'
]
,
 
'
r
e
s
u
l
t
s
'
:
 
r
e
s
p
[
'
r
e
s
u
l
t
s
'
]
}
)
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
}
)


 
 
 
 
 
 
 
 
 
 
 
 
o
b
s
e
r
v
a
t
i
o
n
 
=
 
n
e
x
t
_
o
b
s
e
r
v
a
t
i
o
n

 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
o
b
s
e
r
v
a
t
i
o
n
.
g
e
t
(
'
t
e
r
m
i
n
a
t
e
d
'
)
 
o
r
 
o
b
s
e
r
v
a
t
i
o
n
.
g
e
t
(
'
t
r
u
n
c
a
t
e
d
'
)
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
b
r
e
a
k


 
 
 
 
 
 
 
 
s
c
o
r
e
,
 
d
e
t
a
i
l
s
 
=
 
C
r
a
f
t
e
r
S
c
o
r
e
r
.
s
c
o
r
e
_
e
p
i
s
o
d
e
(
o
b
s
e
r
v
a
t
i
o
n
,
 
l
e
n
(
e
p
i
s
o
d
e
_
r
e
w
a
r
d
s
)
,
 
m
a
x
_
s
t
e
p
s
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
R
o
l
l
o
u
t
R
e
s
p
o
n
s
e
(

 
 
 
 
 
 
 
 
 
 
 
 
r
u
n
_
i
d
=
r
e
q
u
e
s
t
.
r
u
n
_
i
d
,

 
 
 
 
 
 
 
 
 
 
 
 
m
e
t
r
i
c
s
=
R
o
l
l
o
u
t
M
e
t
r
i
c
s
(
o
u
t
c
o
m
e
_
r
e
w
a
r
d
=
s
c
o
r
e
,
 
d
e
t
a
i
l
s
=
d
e
t
a
i
l
s
)
,

 
 
 
 
 
 
 
 
 
 
 
 
t
r
a
c
e
=
N
o
n
e
,
 
 
#
 
S
y
n
t
h
 
r
e
c
o
n
s
t
r
u
c
t
s
 
f
r
o
m
 
i
n
f
e
r
e
n
c
e
 
p
r
o
x
y

 
 
 
 
 
 
 
 
 
 
 
 
t
r
a
c
e
_
c
o
r
r
e
l
a
t
i
o
n
_
i
d
=
p
o
l
i
c
y
_
c
o
n
f
i
g
.
g
e
t
(
'
t
r
a
c
e
_
c
o
r
r
e
l
a
t
i
o
n
_
i
d
'
)
,

 
 
 
 
 
 
 
 
)


 
 
 
 
d
e
f
 
p
r
o
v
i
d
e
_
t
a
s
k
s
e
t
_
d
e
s
c
r
i
p
t
i
o
n
(
)
:

 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
{
'
s
p
l
i
t
s
'
:
 
[
'
t
r
a
i
n
'
,
 
'
t
e
s
t
'
]
}


 
 
 
 
d
e
f
 
p
r
o
v
i
d
e
_
t
a
s
k
_
i
n
s
t
a
n
c
e
s
(
s
e
e
d
s
)
:

 
 
 
 
 
 
 
 
f
o
r
 
s
e
e
d
 
i
n
 
s
e
e
d
s
:

 
 
 
 
 
 
 
 
 
 
 
 
y
i
e
l
d
 
T
a
s
k
I
n
f
o
(

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
t
a
s
k
=
{
'
i
d
'
:
 
A
P
P
_
I
D
,
 
'
n
a
m
e
'
:
 
A
P
P
_
N
A
M
E
}
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
d
a
t
a
s
e
t
=
{
'
i
d
'
:
 
A
P
P
_
I
D
,
 
'
s
p
l
i
t
'
:
 
'
t
r
a
i
n
'
,
 
'
i
n
d
e
x
'
:
 
s
e
e
d
}
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
n
f
e
r
e
n
c
e
=
{
'
t
o
o
l
'
:
 
T
O
O
L
_
N
A
M
E
}
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
l
i
m
i
t
s
=
{
'
m
a
x
_
t
u
r
n
s
'
:
 
5
0
}
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
t
a
s
k
_
m
e
t
a
d
a
t
a
=
{
'
s
e
e
d
'
:
 
s
e
e
d
}
,

 
 
 
 
 
 
 
 
 
 
 
 
)


 
 
 
 
r
e
t
u
r
n
 
c
r
e
a
t
e
_
l
o
c
a
l
_
a
p
i
(
L
o
c
a
l
A
P
I
C
o
n
f
i
g
(

 
 
 
 
 
 
 
 
a
p
p
_
i
d
=
A
P
P
_
I
D
,

 
 
 
 
 
 
 
 
n
a
m
e
=
A
P
P
_
N
A
M
E
,

 
 
 
 
 
 
 
 
d
e
s
c
r
i
p
t
i
o
n
=
f
'
{
A
P
P
_
N
A
M
E
}
 
l
o
c
a
l
 
A
P
I
 
f
o
r
 
V
L
M
 
a
g
e
n
t
 
w
i
t
h
 
i
m
a
g
e
-
o
n
l
y
 
o
b
s
e
r
v
a
t
i
o
n
s
.
'
,

 
 
 
 
 
 
 
 
p
r
o
v
i
d
e
_
t
a
s
k
s
e
t
_
d
e
s
c
r
i
p
t
i
o
n
=
p
r
o
v
i
d
e
_
t
a
s
k
s
e
t
_
d
e
s
c
r
i
p
t
i
o
n
,

 
 
 
 
 
 
 
 
p
r
o
v
i
d
e
_
t
a
s
k
_
i
n
s
t
a
n
c
e
s
=
p
r
o
v
i
d
e
_
t
a
s
k
_
i
n
s
t
a
n
c
e
s
,

 
 
 
 
 
 
 
 
r
o
l
l
o
u
t
=
r
u
n
_
r
o
l
l
o
u
t
,

 
 
 
 
 
 
 
 
c
o
r
s
_
o
r
i
g
i
n
s
=
[
'
*
'
]
,

 
 
 
 
)
)



p
r
i
n
t
(
'
L
o
c
a
l
 
A
P
I
 
f
a
c
t
o
r
y
 
d
e
f
i
n
e
d
'
)

In [ ]:
#
 
S
t
e
p
 
6
:
 
G
E
P
A
 
J
o
b
 
R
u
n
n
e
r


d
e
f
 
r
u
n
_
g
e
p
a
_
j
o
b
(

 
 
 
 
*
,

 
 
 
 
a
p
i
_
k
e
y
:
 
s
t
r
,

 
 
 
 
l
o
c
a
l
_
a
p
i
_
u
r
l
:
 
s
t
r
,

 
 
 
 
l
o
c
a
l
_
a
p
i
_
k
e
y
:
 
s
t
r
,

 
 
 
 
b
a
s
e
l
i
n
e
_
s
y
s
t
e
m
_
p
r
o
m
p
t
:
 
s
t
r
,

)
:

 
 
 
 
"
"
"
R
u
n
 
a
 
G
E
P
A
 
p
r
o
m
p
t
 
o
p
t
i
m
i
z
a
t
i
o
n
 
j
o
b
.
"
"
"

 
 
 
 
c
o
n
f
i
g
_
b
o
d
y
 
=
 
{

 
 
 
 
 
 
 
 
'
p
r
o
m
p
t
_
l
e
a
r
n
i
n
g
'
:
 
{

 
 
 
 
 
 
 
 
 
 
 
 
'
a
l
g
o
r
i
t
h
m
'
:
 
'
g
e
p
a
'
,

 
 
 
 
 
 
 
 
 
 
 
 
'
t
a
s
k
_
a
p
p
_
u
r
l
'
:
 
l
o
c
a
l
_
a
p
i
_
u
r
l
,

 
 
 
 
 
 
 
 
 
 
 
 
'
t
a
s
k
_
a
p
p
_
a
p
i
_
k
e
y
'
:
 
l
o
c
a
l
_
a
p
i
_
k
e
y
,

 
 
 
 
 
 
 
 
 
 
 
 
'
e
n
v
_
n
a
m
e
'
:
 
'
c
r
a
f
t
e
r
'
,

 
 
 
 
 
 
 
 
 
 
 
 
'
i
n
i
t
i
a
l
_
p
r
o
m
p
t
'
:
 
{

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
m
e
s
s
a
g
e
s
'
:
 
[
{
'
r
o
l
e
'
:
 
'
s
y
s
t
e
m
'
,
 
'
o
r
d
e
r
'
:
 
0
,
 
'
p
a
t
t
e
r
n
'
:
 
b
a
s
e
l
i
n
e
_
s
y
s
t
e
m
_
p
r
o
m
p
t
}
]
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
w
i
l
d
c
a
r
d
s
'
:
 
{
}
,

 
 
 
 
 
 
 
 
 
 
 
 
}
,

 
 
 
 
 
 
 
 
 
 
 
 
'
p
o
l
i
c
y
'
:
 
{

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
i
n
f
e
r
e
n
c
e
_
m
o
d
e
'
:
 
'
s
y
n
t
h
_
h
o
s
t
e
d
'
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
m
o
d
e
l
'
:
 
P
O
L
I
C
Y
_
M
O
D
E
L
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
p
r
o
v
i
d
e
r
'
:
 
'
o
p
e
n
a
i
'
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
t
e
m
p
e
r
a
t
u
r
e
'
:
 
0
.
0
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
m
a
x
_
c
o
m
p
l
e
t
i
o
n
_
t
o
k
e
n
s
'
:
 
5
1
2
,

 
 
 
 
 
 
 
 
 
 
 
 
}
,

 
 
 
 
 
 
 
 
 
 
 
 
'
g
e
p
a
'
:
 
{

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
e
n
v
_
n
a
m
e
'
:
 
'
c
r
a
f
t
e
r
'
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
e
v
a
l
u
a
t
i
o
n
'
:
 
{
'
s
e
e
d
s
'
:
 
l
i
s
t
(
r
a
n
g
e
(
3
0
)
)
,
 
'
v
a
l
i
d
a
t
i
o
n
_
s
e
e
d
s
'
:
 
l
i
s
t
(
r
a
n
g
e
(
5
0
,
 
5
6
)
)
}
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
r
o
l
l
o
u
t
'
:
 
{
'
b
u
d
g
e
t
'
:
 
R
O
L
L
O
U
T
_
B
U
D
G
E
T
,
 
'
m
a
x
_
c
o
n
c
u
r
r
e
n
t
'
:
 
3
,
 
'
m
i
n
i
b
a
t
c
h
_
s
i
z
e
'
:
 
3
}
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
m
u
t
a
t
i
o
n
'
:
 
{
'
r
a
t
e
'
:
 
0
.
3
}
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
p
o
p
u
l
a
t
i
o
n
'
:
 
{
'
i
n
i
t
i
a
l
_
s
i
z
e
'
:
 
3
,
 
'
n
u
m
_
g
e
n
e
r
a
t
i
o
n
s
'
:
 
N
U
M
_
G
E
N
E
R
A
T
I
O
N
S
,
 
'
c
h
i
l
d
r
e
n
_
p
e
r
_
g
e
n
e
r
a
t
i
o
n
'
:
 
2
}
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
a
r
c
h
i
v
e
'
:
 
{
'
s
i
z
e
'
:
 
5
,
 
'
p
a
r
e
t
o
_
s
e
t
_
s
i
z
e
'
:
 
1
0
}
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
t
o
k
e
n
'
:
 
{
'
m
a
x
_
l
i
m
i
t
'
:
 
4
0
0
0
,
 
'
c
o
u
n
t
i
n
g
_
m
o
d
e
l
'
:
 
'
g
p
t
-
4
'
,
 
'
m
a
x
_
s
p
e
n
d
_
u
s
d
'
:
 
5
0
.
0
}
,

 
 
 
 
 
 
 
 
 
 
 
 
}
,

 
 
 
 
 
 
 
 
 
 
 
 
'
e
n
v
'
:
 
{

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
m
a
x
_
t
u
r
n
s
'
:
 
2
0
,
 
 
#
 
C
a
p
 
V
L
M
 
c
a
l
l
s
 
p
e
r
 
r
o
l
l
o
u
t

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
m
a
x
_
s
t
e
p
s
_
p
e
r
_
e
p
i
s
o
d
e
'
:
 
2
0
0
,

 
 
 
 
 
 
 
 
 
 
 
 
}
,

 
 
 
 
 
 
 
 
 
 
 
 
'
v
e
r
i
f
i
e
r
'
:
 
{

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
e
n
a
b
l
e
d
'
:
 
F
a
l
s
e
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
r
e
w
a
r
d
_
s
o
u
r
c
e
'
:
 
'
t
a
s
k
_
a
p
p
'
,

 
 
 
 
 
 
 
 
 
 
 
 
}
,

 
 
 
 
 
 
 
 
}
,

 
 
 
 
}


 
 
 
 
j
o
b
 
=
 
P
r
o
m
p
t
L
e
a
r
n
i
n
g
J
o
b
.
f
r
o
m
_
d
i
c
t
(

 
 
 
 
 
 
 
 
c
o
n
f
i
g
_
d
i
c
t
=
c
o
n
f
i
g
_
b
o
d
y
,

 
 
 
 
 
 
 
 
b
a
c
k
e
n
d
_
u
r
l
=
S
Y
N
T
H
_
A
P
I
_
B
A
S
E
,

 
 
 
 
 
 
 
 
a
p
i
_
k
e
y
=
a
p
i
_
k
e
y
,

 
 
 
 
 
 
 
 
t
a
s
k
_
a
p
p
_
a
p
i
_
k
e
y
=
l
o
c
a
l
_
a
p
i
_
k
e
y
,

 
 
 
 
 
 
 
 
s
k
i
p
_
h
e
a
l
t
h
_
c
h
e
c
k
=
T
r
u
e
,

 
 
 
 
)

 
 
 
 
j
o
b
_
i
d
 
=
 
j
o
b
.
s
u
b
m
i
t
(
)

 
 
 
 
p
r
i
n
t
(
f
'
G
E
P
A
 
j
o
b
 
c
r
e
a
t
e
d
:
 
{
j
o
b
_
i
d
}
'
)

 
 
 
 

 
 
 
 
r
e
s
u
l
t
 
=
 
j
o
b
.
p
o
l
l
_
u
n
t
i
l
_
c
o
m
p
l
e
t
e
(
t
i
m
e
o
u
t
=
3
6
0
0
.
0
,
 
i
n
t
e
r
v
a
l
=
3
.
0
,
 
p
r
o
g
r
e
s
s
=
T
r
u
e
)

 
 
 
 
p
r
i
n
t
(
f
'
G
E
P
A
 
j
o
b
 
f
i
n
i
s
h
e
d
:
 
{
r
e
s
u
l
t
.
s
t
a
t
u
s
.
v
a
l
u
e
}
'
)

 
 
 
 
r
e
t
u
r
n
 
r
e
s
u
l
t



p
r
i
n
t
(
'
G
E
P
A
 
j
o
b
 
r
u
n
n
e
r
 
d
e
f
i
n
e
d
'
)

In [ ]:
#
 
S
t
e
p
 
7
:
 
E
v
a
l
 
J
o
b
 
R
u
n
n
e
r


E
V
A
L
_
M
O
D
E
L
 
=
 
"
g
p
t
-
4
o
-
m
i
n
i
"
 
 
#
 
U
s
e
 
r
e
a
l
 
O
p
e
n
A
I
 
m
o
d
e
l
 
f
o
r
 
e
v
a
l
 
(
e
v
a
l
 
d
o
e
s
n
'
t
 
s
u
p
p
o
r
t
 
s
y
n
t
h
_
h
o
s
t
e
d
)

O
P
E
N
A
I
_
A
P
I
_
K
E
Y
 
=
 
o
s
.
e
n
v
i
r
o
n
.
g
e
t
(
'
O
P
E
N
A
I
_
A
P
I
_
K
E
Y
'
,
 
'
'
)
 
 
#
 
G
e
t
 
O
p
e
n
A
I
 
k
e
y
 
f
o
r
 
e
v
a
l
 
j
o
b
s


d
e
f
 
r
u
n
_
e
v
a
l
_
j
o
b
(
*
,
 
l
o
c
a
l
_
a
p
i
_
u
r
l
:
 
s
t
r
,
 
l
o
c
a
l
_
a
p
i
_
k
e
y
:
 
s
t
r
,
 
s
e
e
d
s
:
 
l
i
s
t
[
i
n
t
]
,
 
m
o
d
e
:
 
s
t
r
)
:

 
 
 
 
"
"
"
R
u
n
 
a
n
 
e
v
a
l
 
j
o
b
 
a
n
d
 
w
a
i
t
 
f
o
r
 
c
o
m
p
l
e
t
i
o
n
.
"
"
"

 
 
 
 
c
o
n
f
i
g
 
=
 
E
v
a
l
J
o
b
C
o
n
f
i
g
(

 
 
 
 
 
 
 
 
t
a
s
k
_
a
p
p
_
u
r
l
=
l
o
c
a
l
_
a
p
i
_
u
r
l
,

 
 
 
 
 
 
 
 
b
a
c
k
e
n
d
_
u
r
l
=
S
Y
N
T
H
_
A
P
I
_
B
A
S
E
,

 
 
 
 
 
 
 
 
a
p
i
_
k
e
y
=
S
Y
N
T
H
_
A
P
I
_
K
E
Y
,

 
 
 
 
 
 
 
 
t
a
s
k
_
a
p
p
_
a
p
i
_
k
e
y
=
l
o
c
a
l
_
a
p
i
_
k
e
y
,

 
 
 
 
 
 
 
 
e
n
v
_
n
a
m
e
=
'
c
r
a
f
t
e
r
'
,

 
 
 
 
 
 
 
 
s
e
e
d
s
=
s
e
e
d
s
,

 
 
 
 
 
 
 
 
p
o
l
i
c
y
_
c
o
n
f
i
g
=
{

 
 
 
 
 
 
 
 
 
 
 
 
'
m
o
d
e
l
'
:
 
E
V
A
L
_
M
O
D
E
L
,

 
 
 
 
 
 
 
 
 
 
 
 
'
p
r
o
v
i
d
e
r
'
:
 
'
o
p
e
n
a
i
'
,

 
 
 
 
 
 
 
 
 
 
 
 
'
a
p
i
_
k
e
y
'
:
 
O
P
E
N
A
I
_
A
P
I
_
K
E
Y
,
 
 
#
 
P
a
s
s
 
O
p
e
n
A
I
 
k
e
y
 
t
o
 
t
a
s
k
 
a
p
p

 
 
 
 
 
 
 
 
}
,

 
 
 
 
 
 
 
 
e
n
v
_
c
o
n
f
i
g
=
{

 
 
 
 
 
 
 
 
 
 
 
 
'
m
a
x
_
s
t
e
p
s
_
p
e
r
_
e
p
i
s
o
d
e
'
:
 
2
0
0
,

 
 
 
 
 
 
 
 
 
 
 
 
'
m
a
x
_
t
u
r
n
s
'
:
 
2
0
,

 
 
 
 
 
 
 
 
}
,

 
 
 
 
 
 
 
 
c
o
n
c
u
r
r
e
n
c
y
=
5
,

 
 
 
 
)

 
 
 
 
j
o
b
 
=
 
E
v
a
l
J
o
b
(
c
o
n
f
i
g
)

 
 
 
 
j
o
b
_
i
d
 
=
 
j
o
b
.
s
u
b
m
i
t
(
)

 
 
 
 
p
r
i
n
t
(
f
'
 
 
{
m
o
d
e
}
 
e
v
a
l
 
j
o
b
:
 
{
j
o
b
_
i
d
}
'
)

 
 
 
 
r
e
t
u
r
n
 
j
o
b
.
p
o
l
l
_
u
n
t
i
l
_
c
o
m
p
l
e
t
e
(
t
i
m
e
o
u
t
=
6
0
0
.
0
,
 
i
n
t
e
r
v
a
l
=
2
.
0
,
 
p
r
o
g
r
e
s
s
=
T
r
u
e
)



p
r
i
n
t
(
'
E
v
a
l
 
j
o
b
 
r
u
n
n
e
r
 
d
e
f
i
n
e
d
'
)

In [ ]:
ENVIRONMENT_API_KEY = ensure_localapi_auth(
    backend_base=SYNTH_API_BASE,
    synth_api_key=SYNTH_API_KEY,
)
print(f'Env key ready: {ENVIRONMENT_API_KEY[:12]}...{ENVIRONMENT_API_KEY[-4:]}')


In [ ]:
# Step 9: Define Baseline Prompt

allowed_actions = ', '.join(CRAFTER_ALLOWED_ACTIONS)
baseline_prompt = (
    'You are an agent playing Crafter, a survival crafting game. '
    'Your goal is to survive and unlock achievements by exploring, crafting, and building. '
    'You can see the game state through images. Analyze each image carefully to understand '
    'your surroundings, inventory, health, and available resources. '
    'Use the crafter_interact tool to execute actions. '
    "Key mechanics: use 'do' only when adjacent to a resource (tree, stone, cow, plant); "
    'it does nothing on grass or water. '
    'Craft progression: wood -> table -> wood_pickaxe -> stone -> stone_pickaxe -> iron tools. '
    'Sleep when energy is low to restore and unlock wake_up. '
    f'Available actions: {allowed_actions}. '
    'Only use these action names and return 2-5 actions per decision. '
    'Strategy: move toward trees to collect wood; place a table once you have wood; '
    'craft a wood pickaxe, then collect stone and craft a stone pickaxe; '
    'progress toward iron tools and combat when safe.'
)

print('Baseline prompt:')
print(baseline_prompt[:200] + '...')

In [ ]:
#
 
S
t
e
p
 
1
0
:
 
S
t
a
r
t
 
B
a
s
e
l
i
n
e
 
L
o
c
a
l
 
A
P
I


b
a
s
e
l
i
n
e
_
a
p
p
 
=
 
c
r
e
a
t
e
_
c
r
a
f
t
e
r
_
v
l
m
_
l
o
c
a
l
_
a
p
i
(
b
a
s
e
l
i
n
e
_
p
r
o
m
p
t
,
 
e
n
v
i
r
o
n
m
e
n
t
_
a
p
i
_
k
e
y
)

r
u
n
_
s
e
r
v
e
r
_
b
a
c
k
g
r
o
u
n
d
(
b
a
s
e
l
i
n
e
_
a
p
p
,
 
p
o
r
t
=
L
O
C
A
L
_
A
P
I
_
P
O
R
T
)

a
w
a
i
t
 
w
a
i
t
_
f
o
r
_
h
e
a
l
t
h
_
c
h
e
c
k
(
'
1
2
7
.
0
.
0
.
1
'
,
 
L
O
C
A
L
_
A
P
I
_
P
O
R
T
,
 
e
n
v
i
r
o
n
m
e
n
t
_
a
p
i
_
k
e
y
,
 
t
i
m
e
o
u
t
=
6
0
.
0
)


i
f
 
U
S
E
_
T
U
N
N
E
L
:

 
 
 
 
#
 
C
r
e
a
t
e
 
t
u
n
n
e
l
 
t
o
 
e
x
p
o
s
e
 
l
o
c
a
l
 
A
P
I
 
t
o
 
t
h
e
 
i
n
t
e
r
n
e
t

 
 
 
 
p
r
i
n
t
(
f
'
C
r
e
a
t
i
n
g
 
t
u
n
n
e
l
 
f
o
r
 
p
o
r
t
 
{
L
O
C
A
L
_
A
P
I
_
P
O
R
T
}
.
.
.
'
)

 
 
 
 
b
a
s
e
l
i
n
e
_
t
u
n
n
e
l
 
=
 
a
w
a
i
t
 
T
u
n
n
e
l
e
d
L
o
c
a
l
A
P
I
.
c
r
e
a
t
e
(

 
 
 
 
 
 
 
 
l
o
c
a
l
_
p
o
r
t
=
L
O
C
A
L
_
A
P
I
_
P
O
R
T
,

 
 
 
 
 
 
 
 
b
a
c
k
e
n
d
=
T
u
n
n
e
l
B
a
c
k
e
n
d
.
C
l
o
u
d
f
l
a
r
e
M
a
n
a
g
e
d
T
u
n
n
e
l
,

 
 
 
 
 
 
 
 
a
p
i
_
k
e
y
=
S
Y
N
T
H
_
A
P
I
_
K
E
Y
,

 
 
 
 
 
 
 
 
e
n
v
_
a
p
i
_
k
e
y
=
e
n
v
i
r
o
n
m
e
n
t
_
a
p
i
_
k
e
y
,

 
 
 
 
 
 
 
 
b
a
c
k
e
n
d
_
u
r
l
=
S
Y
N
T
H
_
A
P
I
_
B
A
S
E
,

 
 
 
 
 
 
 
 
p
r
o
g
r
e
s
s
=
T
r
u
e
,

 
 
 
 
)

 
 
 
 
B
A
S
E
L
I
N
E
_
L
O
C
A
L
_
A
P
I
_
U
R
L
 
=
 
b
a
s
e
l
i
n
e
_
t
u
n
n
e
l
.
u
r
l

e
l
s
e
:

 
 
 
 
B
A
S
E
L
I
N
E
_
L
O
C
A
L
_
A
P
I
_
U
R
L
 
=
 
f
'
h
t
t
p
:
/
/
l
o
c
a
l
h
o
s
t
:
{
L
O
C
A
L
_
A
P
I
_
P
O
R
T
}
'


p
r
i
n
t
(
f
'
B
a
s
e
l
i
n
e
 
l
o
c
a
l
 
A
P
I
 
U
R
L
:
 
{
B
A
S
E
L
I
N
E
_
L
O
C
A
L
_
A
P
I
_
U
R
L
}
'
)

In [ ]:
# Step 11: Run GEPA Optimization

print('Starting GEPA optimization...')
print(f'  Rollout budget: {ROLLOUT_BUDGET}')
print(f'  Generations: {NUM_GENERATIONS}')

job_result = run_gepa_job(
    api_key=SYNTH_API_KEY,
    local_api_url=BASELINE_LOCAL_API_URL,
    baseline_system_prompt=baseline_prompt,
)

print(f'\nGEPA Status: {job_result.status.value}')
if job_result.succeeded:
    print('GEPA optimization succeeded!')
else:
    print(f'GEPA failed: {job_result.error}')

In [ ]:
# Step 12: Extract Optimized Prompt

def extract_system_prompt(best_prompt: Optional[Dict[str, Any]]) -> Optional[str]:
    """Extract system prompt from prompt learning results."""
    if not best_prompt:
        return None
    for msg in best_prompt.get('messages', []):
        if msg.get('role') == 'system':
            return msg.get('pattern') or msg.get('content')
    for sec in best_prompt.get('sections', []):
        if sec.get('role') == 'system':
            return sec.get('content')
    return None

optimized_prompt = None

if job_result.succeeded:
    pl_client = PromptLearningClient(SYNTH_API_BASE, SYNTH_API_KEY)
    prompt_results = await pl_client.get_prompts(job_result.job_id)
    optimized_prompt = extract_system_prompt(prompt_results.best_prompt)
    
    if optimized_prompt:
        print('=' * 60)
        print('OPTIMIZED PROMPT')
        print('=' * 60)
        print(optimized_prompt[:800] + '...' if len(optimized_prompt) > 800 else optimized_prompt)
        print('=' * 60)
        
        # Save to results directory
        results_dir = Path('results')
        results_dir.mkdir(exist_ok=True)
        with open(results_dir / 'optimized_prompt.txt', 'w') as f:
            f.write(optimized_prompt)
        print(f'\nSaved optimized prompt to: {results_dir / "optimized_prompt.txt"}')
    else:
        print('Failed to extract optimized prompt from results')
else:
    print('Skipping prompt extraction (GEPA did not succeed)')

In [ ]:
#
 
S
t
e
p
 
1
3
:
 
R
u
n
 
E
v
a
l
u
a
t
i
o
n
 
(
B
a
s
e
l
i
n
e
 
v
s
 
O
p
t
i
m
i
z
e
d
)


E
V
A
L
_
S
E
E
D
S
 
=
 
l
i
s
t
(
r
a
n
g
e
(
1
0
0
,
 
1
2
0
)
)
 
 
#
 
2
0
 
h
e
l
d
-
o
u
t
 
t
e
s
t
 
s
a
m
p
l
e
s


i
f
 
o
p
t
i
m
i
z
e
d
_
p
r
o
m
p
t
:

 
 
 
 
#
 
S
t
a
r
t
 
o
p
t
i
m
i
z
e
d
 
l
o
c
a
l
 
A
P
I

 
 
 
 
o
p
t
i
m
i
z
e
d
_
a
p
p
 
=
 
c
r
e
a
t
e
_
c
r
a
f
t
e
r
_
v
l
m
_
l
o
c
a
l
_
a
p
i
(
o
p
t
i
m
i
z
e
d
_
p
r
o
m
p
t
,
 
e
n
v
i
r
o
n
m
e
n
t
_
a
p
i
_
k
e
y
)

 
 
 
 
r
u
n
_
s
e
r
v
e
r
_
b
a
c
k
g
r
o
u
n
d
(
o
p
t
i
m
i
z
e
d
_
a
p
p
,
 
p
o
r
t
=
O
P
T
I
M
I
Z
E
D
_
L
O
C
A
L
_
A
P
I
_
P
O
R
T
)

 
 
 
 
a
w
a
i
t
 
w
a
i
t
_
f
o
r
_
h
e
a
l
t
h
_
c
h
e
c
k
(
'
1
2
7
.
0
.
0
.
1
'
,
 
O
P
T
I
M
I
Z
E
D
_
L
O
C
A
L
_
A
P
I
_
P
O
R
T
,
 
e
n
v
i
r
o
n
m
e
n
t
_
a
p
i
_
k
e
y
,
 
t
i
m
e
o
u
t
=
6
0
.
0
)

 
 
 
 

 
 
 
 
i
f
 
U
S
E
_
T
U
N
N
E
L
:

 
 
 
 
 
 
 
 
#
 
C
r
e
a
t
e
 
t
u
n
n
e
l
 
f
o
r
 
o
p
t
i
m
i
z
e
d
 
A
P
I

 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
'
C
r
e
a
t
i
n
g
 
t
u
n
n
e
l
 
f
o
r
 
p
o
r
t
 
{
O
P
T
I
M
I
Z
E
D
_
L
O
C
A
L
_
A
P
I
_
P
O
R
T
}
.
.
.
'
)

 
 
 
 
 
 
 
 
o
p
t
i
m
i
z
e
d
_
t
u
n
n
e
l
 
=
 
a
w
a
i
t
 
T
u
n
n
e
l
e
d
L
o
c
a
l
A
P
I
.
c
r
e
a
t
e
(

 
 
 
 
 
 
 
 
 
 
 
 
l
o
c
a
l
_
p
o
r
t
=
O
P
T
I
M
I
Z
E
D
_
L
O
C
A
L
_
A
P
I
_
P
O
R
T
,

 
 
 
 
 
 
 
 
 
 
 
 
b
a
c
k
e
n
d
=
T
u
n
n
e
l
B
a
c
k
e
n
d
.
C
l
o
u
d
f
l
a
r
e
M
a
n
a
g
e
d
T
u
n
n
e
l
,

 
 
 
 
 
 
 
 
 
 
 
 
a
p
i
_
k
e
y
=
S
Y
N
T
H
_
A
P
I
_
K
E
Y
,

 
 
 
 
 
 
 
 
 
 
 
 
e
n
v
_
a
p
i
_
k
e
y
=
e
n
v
i
r
o
n
m
e
n
t
_
a
p
i
_
k
e
y
,

 
 
 
 
 
 
 
 
 
 
 
 
b
a
c
k
e
n
d
_
u
r
l
=
S
Y
N
T
H
_
A
P
I
_
B
A
S
E
,

 
 
 
 
 
 
 
 
 
 
 
 
p
r
o
g
r
e
s
s
=
T
r
u
e
,

 
 
 
 
 
 
 
 
)

 
 
 
 
 
 
 
 
O
P
T
I
M
I
Z
E
D
_
L
O
C
A
L
_
A
P
I
_
U
R
L
 
=
 
o
p
t
i
m
i
z
e
d
_
t
u
n
n
e
l
.
u
r
l

 
 
 
 
e
l
s
e
:

 
 
 
 
 
 
 
 
O
P
T
I
M
I
Z
E
D
_
L
O
C
A
L
_
A
P
I
_
U
R
L
 
=
 
f
'
h
t
t
p
:
/
/
l
o
c
a
l
h
o
s
t
:
{
O
P
T
I
M
I
Z
E
D
_
L
O
C
A
L
_
A
P
I
_
P
O
R
T
}
'

 
 
 
 

 
 
 
 
p
r
i
n
t
(
f
'
O
p
t
i
m
i
z
e
d
 
l
o
c
a
l
 
A
P
I
 
U
R
L
:
 
{
O
P
T
I
M
I
Z
E
D
_
L
O
C
A
L
_
A
P
I
_
U
R
L
}
'
)

 
 
 
 
p
r
i
n
t
(
f
'
\
n
R
u
n
n
i
n
g
 
e
v
a
l
u
a
t
i
o
n
 
o
n
 
{
l
e
n
(
E
V
A
L
_
S
E
E
D
S
)
}
 
s
e
e
d
s
.
.
.
'
)


 
 
 
 
#
 
R
u
n
 
b
a
s
e
l
i
n
e
 
e
v
a
l

 
 
 
 
p
r
i
n
t
(
'
\
n
R
u
n
n
i
n
g
 
B
A
S
E
L
I
N
E
 
e
v
a
l
.
.
.
'
)

 
 
 
 
b
a
s
e
l
i
n
e
_
e
v
a
l
 
=
 
r
u
n
_
e
v
a
l
_
j
o
b
(

 
 
 
 
 
 
 
 
l
o
c
a
l
_
a
p
i
_
u
r
l
=
B
A
S
E
L
I
N
E
_
L
O
C
A
L
_
A
P
I
_
U
R
L
,

 
 
 
 
 
 
 
 
l
o
c
a
l
_
a
p
i
_
k
e
y
=
e
n
v
i
r
o
n
m
e
n
t
_
a
p
i
_
k
e
y
,

 
 
 
 
 
 
 
 
s
e
e
d
s
=
E
V
A
L
_
S
E
E
D
S
,

 
 
 
 
 
 
 
 
m
o
d
e
=
'
b
a
s
e
l
i
n
e
'
,

 
 
 
 
)


 
 
 
 
#
 
R
u
n
 
o
p
t
i
m
i
z
e
d
 
e
v
a
l

 
 
 
 
p
r
i
n
t
(
'
\
n
R
u
n
n
i
n
g
 
O
P
T
I
M
I
Z
E
D
 
e
v
a
l
.
.
.
'
)

 
 
 
 
o
p
t
i
m
i
z
e
d
_
e
v
a
l
 
=
 
r
u
n
_
e
v
a
l
_
j
o
b
(

 
 
 
 
 
 
 
 
l
o
c
a
l
_
a
p
i
_
u
r
l
=
O
P
T
I
M
I
Z
E
D
_
L
O
C
A
L
_
A
P
I
_
U
R
L
,

 
 
 
 
 
 
 
 
l
o
c
a
l
_
a
p
i
_
k
e
y
=
e
n
v
i
r
o
n
m
e
n
t
_
a
p
i
_
k
e
y
,

 
 
 
 
 
 
 
 
s
e
e
d
s
=
E
V
A
L
_
S
E
E
D
S
,

 
 
 
 
 
 
 
 
m
o
d
e
=
'
o
p
t
i
m
i
z
e
d
'
,

 
 
 
 
)


 
 
 
 
#
 
D
i
s
p
l
a
y
 
r
e
s
u
l
t
s

 
 
 
 
p
r
i
n
t
(
'
\
n
'
 
+
 
'
=
'
 
*
 
6
0
)

 
 
 
 
p
r
i
n
t
(
'
E
V
A
L
U
A
T
I
O
N
 
R
E
S
U
L
T
S
'
)

 
 
 
 
p
r
i
n
t
(
'
=
'
 
*
 
6
0
)

 
 
 
 
p
r
i
n
t
(
f
'
B
a
s
e
l
i
n
e
:
 
{
b
a
s
e
l
i
n
e
_
e
v
a
l
.
r
a
w
}
'
)

 
 
 
 
p
r
i
n
t
(
f
'
O
p
t
i
m
i
z
e
d
:
 
{
o
p
t
i
m
i
z
e
d
_
e
v
a
l
.
r
a
w
}
'
)

 
 
 
 

 
 
 
 
#
 
S
a
v
e
 
r
e
s
u
l
t
s

 
 
 
 
r
e
s
u
l
t
s
_
d
i
r
 
=
 
P
a
t
h
(
'
r
e
s
u
l
t
s
'
)

 
 
 
 
r
e
s
u
l
t
s
_
d
i
r
.
m
k
d
i
r
(
e
x
i
s
t
_
o
k
=
T
r
u
e
)

 
 
 
 
w
i
t
h
 
o
p
e
n
(
r
e
s
u
l
t
s
_
d
i
r
 
/
 
'
e
v
a
l
_
r
e
s
u
l
t
s
.
j
s
o
n
'
,
 
'
w
'
)
 
a
s
 
f
:

 
 
 
 
 
 
 
 
j
s
o
n
.
d
u
m
p
(
{
'
b
a
s
e
l
i
n
e
'
:
 
b
a
s
e
l
i
n
e
_
e
v
a
l
.
r
a
w
,
 
'
o
p
t
i
m
i
z
e
d
'
:
 
o
p
t
i
m
i
z
e
d
_
e
v
a
l
.
r
a
w
}
,
 
f
,
 
i
n
d
e
n
t
=
2
)

 
 
 
 
p
r
i
n
t
(
f
'
\
n
S
a
v
e
d
 
e
v
a
l
 
r
e
s
u
l
t
s
 
t
o
:
 
{
r
e
s
u
l
t
s
_
d
i
r
 
/
 
"
e
v
a
l
_
r
e
s
u
l
t
s
.
j
s
o
n
"
}
'
)

e
l
s
e
:

 
 
 
 
p
r
i
n
t
(
'
S
k
i
p
p
i
n
g
 
e
v
a
l
u
a
t
i
o
n
 
(
n
o
 
o
p
t
i
m
i
z
e
d
 
p
r
o
m
p
t
)
'
)

In [ ]:
# Step 14: Done
print('Demo complete!')